In [8]:
import transformers
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer

from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.1-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [59]:
from transformers import AutoModelForCausalLM,AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)

prompt = f"""

You will be provided with cricket commentary. Your task is to standardize the commentary by rephrasing it into a formal and concise tone. Remove personal identifiers and replace them with the following special tokens: [BATSMAN], [BOWLER], [FIELDER], [UMPIRE], [VENUE]. Ensure that the commentary remains concise and retains its original meaning.

If no commentary or irrelevant text is provided, output "None". 



"""

messages = [
    {"role": "system", "content": prompt},
    {'role': 'user' , 'content' : "Commentary: "}
]

prompts = tokenizer.apply_chat_template(messages, tokenize=False)

res = tokenizer(prompts, return_tensors="pt")

response = model_nf4.generate(**res, max_new_tokens=150, do_sample=True, temperature=0.7)

response_text = tokenizer.decode(response[0])
print(response_text)

In [ ]:
import re

def extract_assistant_response(text):
    """
    Extracts the assistant's response using regex.
    The response is located between <|start_header_id|>assistant<|end_header_id|> and <|eot_id|>.
    
    Args:
        text (str): The input text containing the conversation log.
        
    Returns:
        str: The extracted response of the assistant, or an empty string if not found.
    """
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract and return the response
        return match.group(1).strip()
    else:
        return ""  # Return empty string if no match is found

model_id = "meta-llama/Llama-3.1-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)



tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token 

prompt = f"""

You will be provided with cricket commentary. Your task is to standardize the commentary by rephrasing it into a formal and concise tone. Remove personal identifiers and replace them with the following special tokens: [BATSMAN], [BOWLER], [FIELDER], [UMPIRE], [VENUE]. Ensure that the commentary remains concise and retains its original meaning.

If no commentary or irrelevant text is provided, output "None". 



"""

messages_batch = [
    [ {"role": "system", "content": prompt}, {"role": "user", "content": "Commentary: First example commentary."}],
    [ {"role": "system", "content": prompt}, {"role": "user", "content": "Commentary: Second example commentary."}],
    # Add more messages as needed
]

text_batch = tokenizer.apply_chat_template(messages_batch, tokenize=False)
inputs_batch = tokenizer(text_batch, return_tensors="pt", padding=True)

generated_ids_batch = model_nf4.generate(
    **inputs_batch,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7
)

response = extract_assistant_response(response)

In [62]:
messages_batch = [
    [ {"role": "system", "content": prompt}, {"role": "user", "content": "Commentary: First example commentary."}],
    [ {"role": "system", "content": prompt}, {"role": "user", "content": "Commentary: Second example commentary."}],
    # Add more messages as needed
]

In [66]:
tokenizer.pad_token = tokenizer.eos_token 

In [68]:
text_batch = tokenizer.apply_chat_template(messages_batch, tokenize=False)
inputs_batch = tokenizer(text_batch, return_tensors="pt", padding=True)

In [70]:
generated_ids_batch = model_nf4.generate(
    **inputs_batch,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [73]:
import re

def extract_assistant_response(text):
    """
    Extracts the assistant's response using regex.
    The response is located between <|start_header_id|>assistant<|end_header_id|> and <|eot_id|>.
    
    Args:
        text (str): The input text containing the conversation log.
        
    Returns:
        str: The extracted response of the assistant, or an empty string if not found.
    """
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract and return the response
        return match.group(1).strip()
    else:
        return ""  # Return empty string if no match is found

In [75]:
response_texts = tokenizer.batch_decode(generated_ids_batch, skip_special_tokens=False)
for response in response_texts:
    print(extract_assistant_response(response))

None
None


In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

text_generation_pipeline = pipeline(
    model=model_nf4,
    tokenizer=tokenizer,
    do_sample=True,
    return_full_text=False,
    task="text-generation",
    temperature=0.1,
    top_p=0.15,
    top_k=0,
    max_new_tokens=150,  # Adjust as needed
    repetition_penalty=1.1
)


In [25]:
text_generation_pipeline(["hi"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[[{'generated_text': ', i\'m a bit confused about the concept of "probability" in statistics. can you help me clarify it?\n\n## Step 1: Understanding Probability Basics\nProbability is a measure of the likelihood that an event will occur. It\'s often represented as a number between 0 and 1, where 0 indicates impossibility and 1 indicates certainty.\n\n## Step 2: Defining Key Terms\n- **Experiment**: Any action or occurrence whose outcome we want to analyze.\n- **Outcome**: The result of an experiment (e.g., flipping a coin).\n- **Sample Space**: The set of all possible outcomes of an experiment.\n- **Event**: A subset of the sample space (e.g., getting heads when flipping a coin).\n\n##'}]]

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import os
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch 

# Function to extract assistant's response
def extract_assistant_response(text):
    """
    Extracts the assistant's response using regex.
    The response is located between <|start_header_id|>assistant<|end_header_id|> and <|eot_id|>.
    """
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Model and tokenizer setup
model_id = "meta-llama/Llama-3.1-8B-Instruct"
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

tokenizer = AutoTokenizer.from_pretrained(model_id,padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
# Prompt for the model
prompt = """
You will be provided with cricket commentary. Your task is to standardize the commentary by rephrasing it into a formal and concise tone. Remove personal identifiers and replace them with the following special tokens: [BATSMAN], [BOWLER], [FIELDER], [UMPIRE], [VENUE]. Ensure that the commentary remains concise and retains its original meaning.

If no commentary or irrelevant text is provided, output "None".
"""

# Function to load and process JSON files
def load_commentaries_from_json(folder_path):
    """
    Load and combine the `text` fields from all JSON files in a folder.
    """
    all_commentaries = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r") as f:
                data = json.load(f)
                # Combine all `text` fields to form one commentary
                commentary = " ".join(item["text"].strip() for item in data if "text" in item)
                all_commentaries.append(commentary)
    return all_commentaries

# Function to perform batch inference
def batch_inference(commentaries, batch_size=4):
    """
    Perform inference in batches.
    """
    results = []
    for i in range(0, len(commentaries), batch_size):
        batch = commentaries[i : i + batch_size]
        # Prepare batch messages
        messages_batch = [
            [{"role": "system", "content": prompt}, {"role": "user", "content": f"Commentary: {c}"}]
            for c in batch
        ]
        for c in batch:
            print(c)
            print()
        # Prepare inputs for the model
        text_batch = tokenizer.apply_chat_template(messages_batch, tokenize=False)
        inputs_batch = tokenizer(text_batch, return_tensors="pt", padding=True)
        inputs_batch = {key: tensor.to(device) for key, tensor in inputs_batch.items()}

        
        # Generate responses
        generated_ids_batch = model_nf4.generate(
            **inputs_batch,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Decode and extract responses
        outputs = tokenizer.batch_decode(generated_ids_batch, skip_special_tokens=False)
        parsed_outs = [extract_assistant_response(output) for output in outputs]
        print('-------------------')
        for i in parsed_outs: 
            print(i)
            print()
        results.extend(parsed_outs)
    return results

# Main function
def process_directory_and_run_inference(folder_path, batch_size=4):
    """
    Process a directory of JSON files and run inference on the extracted commentaries.
    """
    commentaries = load_commentaries_from_json(folder_path)
    results = batch_inference(commentaries, batch_size=batch_size)
    return results



In [18]:

folder_path = "/home/ACG/data/raw/whisperx_transcripts"  # Replace with the actual path
batch_size = 4  # Adjust as needed
standardized_commentaries = process_directory_and_run_inference(folder_path, batch_size=batch_size)

# Print results
for i, result in enumerate(standardized_commentaries):
    print(f"Standardized Commentary {i + 1}: {result}")


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


road end today and it's pleasant to drive there but that's gone through for the third boundary in the last three overs. Yeah an exciting

outside his pads.

to the short pitch bowling to the leg side boundary which i think was wise yeah easy as you like drag down jamie smith has just pumped that straight through mid wicket

A little bit short and played away off the back foot by Harris through the vacant point area down to the fielder on the point boundary and that is the single Harris needed for his 150. Raises his bat.



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


-------------------
None

The [BATSMAN] is caught outside his leg stump.

[BATSMAN] guides the short pitch delivery towards the leg side boundary, a tactical decision that paid off as he scores easily off the over. [BATSMAN] then executes a straight drive through mid wicket for four.

[BATSMAN] played a short and well-timed shot through the vacant point area, scoring a single off the back foot. He reached his 150 and raised his bat in celebration.

You've got to play with a certain ball. It's our home series. As Sean has really played it beautifully down the ground, offering the full face of the bat.

as Ingram belts one through the offside for four just a little bit more flight for Moriarty whose disgruntlement looks to be on the

Fisher in. Bowls. Ingram whips it off his legs. Past the umpire. Sizzling away for four. Latched onto that one very quickly. Turned the wrists on it nicely.

bowls to Lithe who drives, gets a full on half volley and that has been spanked for four. The final


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


-------------------
[BATSMAN] scores a boundary with a square cut offside. The ball would have been a four if it hadn't been stopped just short of the boundary.

[BATSMAN] attempts to drive Fuller's delivery, but the ball is initially dropped at first slip. However, the second slip fielder manages to cling onto the catch, resulting in [BATSMAN]'s dismissal and giving [BOWLER] the wicket, with Surrey claiming their sixth scalp of the day.

[NATIVE] batsman plays a shot to mid wicket on the last ball of the day, ending it on a positive note for the visitors.

[BATSMAN] drives the delivery from [BOWLER] for four, placing it just in front of the [FIELDER] at the bowler's end.

yesterday.

Once again, balls to Reece, that one's down the leg side, it goes all the way past a diving Ben Cox for four runs. He can have an impact, goes in, balls outside.

Straight to backward point. Gregory falls on 80. Not much going on with the delivery. Tried to carve it away. He's done a great job, Gregory.



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


-------------------
None

[BATSMAN] scores four runs as the ball goes past a diving [FIELDER] at the leg side.

[BATSMAN] plays a defensive shot, aiming for the backward point. He scores 80 runs. The delivery was not particularly challenging.

None

Howard stand end and it comes again it's a chance and it's caught by Finlay Bean great catch at third slip from Finlay Bean and again Fisher is right on the money with the line and length again

Thompson then, bowling here to Joe Root. And there it is. Joe Root with a reverse paddle sweep has brought up another superb hundred. It was almost inevitable, really, from early on that he was going to get there. Serene progress throughout, has not given a chance.

That lateral movement through the air. Oh, Bancode comes in again. There's an edge. I think it bounced in front of the slipcord in there. There's three slips and a gully. But it just bounced in front. It was a genuine edge.

27 years of age now. This delivery wide of off stump and Bean h

KeyboardInterrupt: 

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")